<a href="https://colab.research.google.com/github/RianNugroho/tugasANN/blob/master/JST_AANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import math
import numpy as np


def backpropOutput(index_datasets, label, activation, weight, learning_rate, x_in):
    delta = np.zeros((1, len(activation[0])))
    for i in range(len(activation[0])):
        delta[0][i] = ( activation[0][i]-label[index_datasets][i]) * activation[0][i] * (1 - activation[0][i])
    
    for i in range(len(weight) - 1):
        for j in range(len(weight[i])):
            weight[i][j] += learning_rate * delta[0][j] * x_in[0][i]
    for i in range(len(weight[len(weight) - 1])):
        weight[len(weight) - 1][i] = learning_rate * delta[0][i]
    return delta, weight


def backpropHiddenLayer(old_weight,weight, activation, next_err, learning_rate, x_in):
    delta_in=np.zeros((1,6))
    temp=np.zeros((len(weight)-1,len(weight[0])))
    for i in range(len(weight)-1):
        for j in range(len(weight[i])):
            temp[i][j]=weight[i][j]
   
    
    delta_in=np.matmul(next_err,np.transpose(temp))
    weight_correction=learning_rate*delta_in*(1-delta_in)*np.transpose(x_in)
    old_weight+=weight_correction

    return delta_in,old_weight

def sigmoidActivation(activation):
    for k in range(len(activation[0])):
        activation[0][k] = math.tanh(activation[0][k])
    return activation


def lastsigmoidActivation(activation):
    max = 0
    index = 0
    for k in range(len(activation[0])):
        activation[0][k] = math.tanh(activation[0][k])
        if (k == 0):
            max = activation[0][k]
        elif (activation[0][k] > max):
            max = activation[0][k]
            index = k
    for i in range(len(activation[0])):
        if(i!=index):
          activation[0][i]=0

    return activation


def multiply(a, b):
    temp=np.zeros((1,len(a[0])+1))
    for i in range(len(a[0])):
      temp[0][i]=a[0][i]
    temp[0][len(temp[0])-1]=-1
    activation = np.matmul(temp, b)
    return activation


def train(datasets, epoch, learning_rate, label):
    ##jangan lupa taruh bias
    layer1 = np.random.rand(7, 6)
    activation1 = np.zeros((1, 6))
    layer2 = np.random.rand(7, 2)
    activation2 = np.zeros((1, 2))
    layer3 = np.random.rand(3, 6)
    activation3 = np.zeros((1, 6))
    layer4 = np.random.rand(7, 3)
    activation4 = np.zeros((1, 3))
    ##input to mapping layer
    for i in range(epoch):
        for j in range(len(datasets)):
            
            activation1 = multiply(datasets[j], layer1)
            activation1 = sigmoidActivation(activation1)
            print(activation1)
            ##mapping layer to bottleneck layer
            activation2 = multiply(activation1, layer2)
            activation2 = sigmoidActivation(activation2)
            ##bottleneck layer to demapping layer
            activation3 = multiply(activation2, layer3)
            activation3 = sigmoidActivation(activation3)
            ##demapping layer to output layer
            activation4 = np.matmul(activation3, layer4)
            activation4 = sigmoidActivation(activation4)
           
            ##backpropagation

            delta, layer4 = backpropOutput(j, label, activation4, layer4, learning_rate, activation3)
            delta, layer3 = backpropHiddenLayer(layer3,layer4, activation3, delta, learning_rate, activation2)
            delta, layer2 = backpropHiddenLayer(layer2,layer3, activation2, delta, learning_rate, activation1)
            delta, layer1 = backpropHiddenLayer(layer1,layer2, activation1, delta, learning_rate, datasets[j])

    return [layer1, layer2, layer3, layer4]

def predict(datasets,model):

    layer1 = model[0]
    activation1 = np.zeros((1, 6))
    layer2 = model[1]
    activation2 = np.zeros((1, 2))
    layer3 = model[2]
    activation3 = np.zeros((1, 6))
    layer4 = model[3]
    activation4 = np.zeros((1, 3))
    ##input to mapping layer
    temp = np.zeros((1, 7))
    for k in range(len(datasets)):
        temp[0][k] = datasets[k]
    temp[0][len(temp[0])-1] = -1
    activation1 = multiply(temp, layer1)
    activation1 = sigmoidActivation(activation1)
    print("aktivasi 1")
    print(activation1)
    ##mapping layer to bottleneck layer
    activation2 = multiply(activation1, layer2)
    activation2 = sigmoidActivation(activation2)
    print("aktivasi 2")
    print(activation2)
    ##bottleneck layer to demapping layer
    activation3 = multiply(activation2, layer3)
    activation3 = sigmoidActivation(activation3)
    print("aktivasi 3")
    print(activation3)
    ##demapping layer to output layer
    activation4 = np.matmul(activation3, layer4)
    activation4 = sigmoidActivation(activation4)
    print("aktivasi 4")

    return activation4


In [0]:
import pandas as pd

In [0]:
file=pd.read_csv("https://raw.githubusercontent.com/RianNugroho/tugasANN/master/datasetNew.csv")

In [828]:

datasets=[]
for i in range(len(file['target'])):
  temp=[]
  for key in file:
    if(key!='target'):
      temp+=[file[key][i]]
  datasets+=[temp]
min=[0,0,0,0,0,0]
max=[0,0,0,0,0,0]
for i in range(len(datasets)):
  for j in range(len(datasets[i])):
    if(i==0):
      min[j]=datasets[i][j]
      max[j]=datasets[i][j]
    else:
      if(datasets[i][j]<min[j]):
        min[j]=datasets[i][j]
      if(datasets[i][j]>max[j]):
        max[j]=datasets[i][j]
print(min,max)
for i in range(len(datasets)):
  for j in range(len(datasets[i])):
    datasets[i][j]=(datasets[i][j]-min[j])/(max[j]-min[j])

[75148495, 0.05722679, 0.39158708, 0.93986457, 14154961, 0.00327491] [954195095, 0.07792213, 0.50627049, 0.97394237, 211956522, 0.00607186]


In [0]:
label=np.zeros((len(file['target']),3))
for i in range(len(file['target'])):
  if(file['target'][i]==1):
    label[i][0]=1
    label[i][1]=0
    label[i][2]=0
  elif(file['target'][i]==2):
    label[i][0]=0
    label[i][1]=1
    label[i][2]=0
  elif(file['target'][i]==3):
    label[i][0]=0
    label[i][1]=0
    label[i][2]=1


In [830]:
datasets=np.array(datasets)
np.random.shuffle(datasets)
model=train(datasets,500,0.01, label)
print(model)

TypeError: ignored

In [0]:
file=pd.read_csv("https://raw.githubusercontent.com/RianNugroho/tugasANN/master/data_uji.csv")
data_uji=[]
for i in range(len(file['target'])):
  temp=[]
  for key in file:
    if(key!='target'):
      temp+=[file[key][i]]
  data_uji+=[temp]
min=[0,0,0,0,0,0]
max=[0,0,0,0,0,0]
for i in range(len(data_uji)):
  for j in range(len(data_uji[i])):
    if(i==0):
      min[j]=data_uji[i][j]
      max[j]=data_uji[i][j]
    else:
      if(data_uji[i][j]<min[j]):
        min[j]=data_uji[i][j]
      if(data_uji[i][j]>max[j]):
        max[j]=data_uji[i][j]
print(min,max)
for i in range(len(data_uji)):
  for j in range(len(data_uji[i])):
    data_uji[i][j]=(data_uji[i][j]-min[j])/(max[j]-min[j])


In [0]:
data_uji=np.array(data_uji)
for i in range(len(data_uji)):
  print(predict(data_uji[i],model))